### Problem Statement
Flight delays has become a concern issue for airline companies around the world due to the associated financial loses to both aviation industry and passengers. Between 2012 and 2018, around 19% of airline flights from major carriers in United States were delayed affecting the journey of many passengers (Bureau of Transport Statistics, 2018). In this project, we will use machine learning models to predict if the flight will be delayed.

Definition of <b>delayed flight</b>:

- Flight that arrives late at its destination
- If the flight has any delays from its departure, but still arrives to its destination on time, it will not be considered a delayed flight
- Flight that arrive on time is denoted as 0
- Flight that delayed is denoted as 1


### Objectives
- To identify attributes that affect flight delay
- To develop machine learning models that classify flights outcome (delayed or not delayed) with selected attributes 
- To evaluate performance of different machine learning models


### Data Acquisition and Exploration

In [2]:
# Import Dependencies
%matplotlib inline

# Begin Python Imports
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

#### Import data

In [15]:
# Read 2018 airport data 
flight_data = pd.read_csv('flight_data_csv/part-00000-ee0e4b37-8329-45ac-a8a2-d95a2d074432-c000.csv')

# Check first 2 instances and last 2 instances
flight_data.head(2).append(flight_data.tail(2))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2014-01-01,AA,2377,ICT,DFW,1135.0,1144.0,9.0,12.0,1156.0,1253.0,9.0,1300.0,1302.0,2.0,0.0,NaN,0.0,85.0,78.0,57.0,328.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01,AA,2378,MIA,TPA,2225.0,2220.0,-5.0,14.0,2234.0,2311.0,4.0,2325.0,2315.0,-10.0,0.0,NaN,0.0,60.0,55.0,37.0,204.0,NaN,NaN,NaN,NaN,NaN,NaN
299998,2010-01-02,WN,2796,LAS,PDX,1335.0,1422.0,47.0,20.0,1442.0,1644.0,2.0,1605.0,1646.0,41.0,0.0,NaN,0.0,150.0,144.0,122.0,762.0,0.0,0.0,4.0,0.0,37.0,NaN
299999,2010-01-02,WN,5836,LAS,PDX,810.0,810.0,0.0,9.0,819.0,1024.0,4.0,1035.0,1028.0,-7.0,0.0,NaN,0.0,145.0,138.0,125.0,762.0,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
flight_data["CARRIER_DELAY"].value_counts()

0.0      31470
6.0       1505
7.0       1473
5.0       1429
4.0       1419
         ...  
272.0        1
968.0        1
526.0        1
569.0        1
352.0        1
Name: CARRIER_DELAY, Length: 554, dtype: int64

#### Dataset exploration

In [4]:
# Check dimension of dataset
flight_data.shape
print("There are "+ str(flight_data.shape[0]) +" rows and "+ str(flight_data.shape[1]) +" columns from the flight dataset.")

There are 300000 rows and 28 columns from the flight dataset.


In [5]:
# Convert flight_data column to datetime format
flight_data['FL_DATE'] =  pd.to_datetime(flight_data['FL_DATE'], format='%Y-%m-%d')

In [6]:
# Check column type
flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FL_DATE              300000 non-null  datetime64[ns]
 1   OP_CARRIER           300000 non-null  object        
 2   OP_CARRIER_FL_NUM    300000 non-null  int64         
 3   ORIGIN               300000 non-null  object        
 4   DEST                 300000 non-null  object        
 5   CRS_DEP_TIME         300000 non-null  float64       
 6   DEP_TIME             294136 non-null  float64       
 7   DEP_DELAY            294090 non-null  float64       
 8   TAXI_OUT             294005 non-null  float64       
 9   WHEELS_OFF           294005 non-null  float64       
 10  WHEELS_ON            293712 non-null  float64       
 11  TAXI_IN              293712 non-null  float64       
 12  CRS_ARR_TIME         300000 non-null  float64       
 13  ARR_TIME      

#### Metadata

<b>FL_DATE</b> = Date of the Flight<br>
<b>OP_CARRIER</b> = Airline Identifier<br>
<b>OP_CARRIER_FL_NUM</b> = Flight Number<br>
<b>ORIGIN</b> = Starting Airport Code<br>
<b>DEST</b> = Destination Airport Code<br>
<b>CRS_DEP_TIME</b> = Planned Departure Time<br>
<b>DEP_TIME</b> = Actual Departure Time<br>
<b>DEP_DELAY</b> = Total Delay on Departure in minutes<br>
<b>TAXI_OUT</b> = The time duration elapsed between departure from the origin airport gate and wheels off<br>
<b>WHEELS_OFF</b> = The time point that the aircraft's wheels leave the ground<br>
<b>WHEELS_ON</b> = The time point that the aircraft'ss wheels touch on the ground<br>
<b>TAXI_IN</b> = The time duration elapsed between wheels-on and gate arrival at the destination airport<br>
<b>CRS_ARR_TIME</b> = Planned arrival time<br>
<b>ARR_TIME</b> = Actual Arrival Time = ARRIVAL_TIME - SCHEDULED_ARRIVAL<br>
<b>ARR_DELAY</b> = Total Delay on Arrival in minutes<br>
<b>CANCELLED</b> = Flight Cancelled (1 = cancelled)<br>
<b>CANCELLATION_CODE</b> = Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security<br>
<b>DIVERTED</b> = Aircraft landed on different airport that the one scheduled<br>
<b>CRS_ELAPSED_TIME</b> = Planned time amount needed for the flight trip<br>
<b>ACTUAL_ELAPSED_TIME</b> = AIR_TIME+TAXI_IN+TAXI_OUT<br>
<b>AIR_TIME</b> = The time duration between wheels_off and wheels_on time<br>
<b>DISTANCE</b> = Distance between two airports<br>
<b>CARRIER_DELAY</b> = Delay caused by the airline in minutes<br>
<b>WEATHER_DELAY</b> = Delay caused by weather<br>
<b>NAS_DELAY</b> = Delay caused by air system<br>
<b>SECURITY_DELAY</b> = caused by security reasons<br>
<b>LATE_AIRCRAFT_DELAY</b> = Delay caused by security<br>

### Data Preprocessing

* [Handle of missing data](#Handle-of-missing-data)
* [Transformation of Duration time taken](#Transformation-of-Duration-time-taken)
* [Rename of OP_CARRIER](#Rename-of-OP_CARRIER)
* [Transformation of Date](#Transformation-of-Date)
* [Data reduction: Focus on Busy Airports](#Data-reduction:-Focus-on-Busy-Airports)
* [Rename of airports and city](#Rename-of-airports-and-city)
* [Transformation of Target attribute: Flight status](#Transformation-of-Target-attribute:-Flight-status)

#### Handle of missing data

In [7]:
# Calculate the proportion of missing data

def checkMissing(data,perc=0):
    """ 
    Takes in a dataframe and returns
    the percentage of missing value.
    """
    missing = [(i, data[i].isna().mean()*100) for i in data]
    missing = pd.DataFrame(missing, columns=["column_name", "percentage"])
    missing = missing[missing.percentage > perc]
    print(missing.sort_values("percentage", ascending=False).reset_index(drop=True))

print("Proportion of missing data in columns")
checkMissing(flight_data)

Proportion of missing data in columns
            column_name  percentage
0           Unnamed: 27  100.000000
1     CANCELLATION_CODE   97.976333
2         CARRIER_DELAY   74.867000
3         WEATHER_DELAY   74.867000
4             NAS_DELAY   74.867000
5        SECURITY_DELAY   74.867000
6   LATE_AIRCRAFT_DELAY   74.867000
7             ARR_DELAY    2.272667
8   ACTUAL_ELAPSED_TIME    2.267000
9              AIR_TIME    2.267000
10            WHEELS_ON    2.096000
11              TAXI_IN    2.096000
12             ARR_TIME    2.096000
13             TAXI_OUT    1.998333
14           WHEELS_OFF    1.998333
15            DEP_DELAY    1.970000
16             DEP_TIME    1.954667


These columns are dropped: `Unnamed: 27`, `CANCELLATION_CODE`, `LATE_AIRCRAFT_DELAY`, `CARRIER_DELAY`, `WEATHER_DELAY`, `NAS_DELAY` and `SECURITY_DELAY` from the dataframe as these columns had more than 50% of missing values which does not provide any useful information to our analysis. Since our main objective is to predict flight delay, column `CANCELLED` is irrelevant to our scope of work as a canceled flight is not a delayed flight as it never left nor arrived to its destination for whatever reason. `OP_CARRIER_FL_NUM` is also not relevant to our predictions. 

After further investigation on the dataset,  `LATE_AIRCRAFT_DELAY`, `CARRIER_DELAY`, `WEATHER_DELAY`, `NAS_DELAY` and `SECURITY_DELAY` are showing the split of delayed minutes for the flight. Thus, the missing data is actually implying that there is no delay for the instance. The missing values for these attributes are imputed with zero.

In [16]:
#Fill the missing value 0
flight_data['LATE_AIRCRAFT_DELAY']=flight_data['LATE_AIRCRAFT_DELAY'].fillna(0)
flight_data['CARRIER_DELWHEELS_ONAY']=flight_data['CARRIER_DELAY'].fillna(0)
flight_data['WEATHER_DELAY']=flight_data['WEATHER_DELAY'].fillna(0)
flight_data['NAS_DELAY']=flight_data['NAS_DELAY'].fillna(0)
flight_data['SECURITY_DELAY']=flight_data['SECURITY_DELAY'].fillna(0)

#### Transformation of Duration time taken
Checking on the calculation for Departure Delay and Arrival Delay in minutes

The derivation of the delayed time of departure and delayed time of arrival is as shown below:
- DEP_DELAY = CRS_DEP_TIME - DEP_TIME
- ARR_DELAY = ARR_TIME - CRS_ARR_TIME

`CRS_DEP_TIME`, `DEP_TIME`, `CRS_ARR_TIME` and `ARR_TIME` can be eliminated as`DEP_DELAY` and `ARR_DELAY` are informative enough to provide the length of time of the delay in minutes.

In [17]:
flight_data[["DEP_TIME","CRS_DEP_TIME","CRS_ARR_TIME","WHEELS_OFF","WHEELS_ON"]]


,DEP_TIME,CRS_DEP_TIME,CRS_ARR_TIME,WHEELS_OFF,WHEELS_ON
0,1144.0,1135.0,1300.0,1156.0,1253.0
1,2220.0,2225.0,2325.0,2234.0,2311.0
2,NaN,2105.0,2205.0,NaN,NaN
3,1805.0,1655.0,1900.0,1811.0,1941.0
4,1440.0,1320.0,1625.0,1449.0,1730.0
...,...,...,...,...,...
299995,2036.0,1925.0,2025.0,2044.0,2116.0
299996,1005.0,950.0,1045.0,1021.0,1053.0
299997,2020.0,1940.0,2205.0,2031.0,2232.0
299998,1422.0,1335.0,1605.0,1442.0,1644.0


In [10]:
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
        # return pd.to_datetime(heure, format='%H:%M:%S')
    
    # pd.to_datetime(heure, format='%Y-%m-%d')

In [18]:
flight_data['DEP_TIME'] = flight_data['DEP_TIME'].apply(format_heure)
flight_data['CRS_DEP_TIME'] = flight_data['CRS_DEP_TIME'].apply(format_heure)

flight_data['ARR_TIME'] = flight_data['ARR_TIME'].apply(format_heure) 
flight_data['CRS_ARR_TIME'] = flight_data['CRS_ARR_TIME'].apply(format_heure)

flight_data['WHEELS_OFF'] = flight_data['WHEELS_OFF'].apply(format_heure)
flight_data['WHEELS_ON'] = flight_data['WHEELS_ON'].apply(format_heure)

In [20]:
flight_data[["DEP_TIME","CRS_DEP_TIME","CRS_ARR_TIME","WHEELS_OFF","WHEELS_ON"]]


,DEP_TIME,CRS_DEP_TIME,CRS_ARR_TIME,WHEELS_OFF,WHEELS_ON
0,11:44:00,11:35:00,13:00:00,11:56:00,12:53:00
1,22:20:00,22:25:00,23:25:00,22:34:00,23:11:00
2,NaN,21:05:00,22:05:00,NaN,NaN
3,18:05:00,16:55:00,19:00:00,18:11:00,19:41:00
4,14:40:00,13:20:00,16:25:00,14:49:00,17:30:00
...,...,...,...,...,...
299995,20:36:00,19:25:00,20:25:00,20:44:00,21:16:00
299996,10:05:00,09:50:00,10:45:00,10:21:00,10:53:00
299997,20:20:00,19:40:00,22:05:00,20:31:00,22:32:00
299998,14:22:00,13:35:00,16:05:00,14:42:00,16:44:00


Obtain duration of time taken for:
- Flight take off and wheels off
- Flight on land and wheels on

In [21]:
# Convert flight_data column to datetime format

def time_difference(actual,plan):
    actual_time = pd.to_timedelta(actual.astype(str)) 
    plan_time = pd.to_timedelta(plan.astype(str))
    return actual_time.sub(plan_time).dt.total_seconds().div(60)

flight_data['WHEELS_OFF_elapse'] =  time_difference(flight_data['WHEELS_OFF'],flight_data['DEP_TIME'])
flight_data['WHEELS_ON_elapse'] =  time_difference(flight_data['ARR_TIME'] ,flight_data['WHEELS_ON'])

# Remove incorrect record
flight_data=flight_data[flight_data['WHEELS_OFF_elapse']>0]
flight_data=flight_data[flight_data['WHEELS_ON_elapse']>0]

In [22]:
# Check first 2 instances and last 2 instances
flight_data.head(2).append(flight_data.tail(2))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,CARRIER_DELWHEELS_ONAY,WHEELS_OFF_elapse,WHEELS_ON_elapse
0,2014-01-01,AA,2377,ICT,DFW,11:35:00,11:44:00,9.0,12.0,11:56:00,12:53:00,9.0,13:00:00,13:02:00,2.0,0.0,NaN,0.0,85.0,78.0,57.0,328.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,12.0,9.0
1,2014-01-01,AA,2378,MIA,TPA,22:25:00,22:20:00,-5.0,14.0,22:34:00,23:11:00,4.0,23:25:00,23:15:00,-10.0,0.0,NaN,0.0,60.0,55.0,37.0,204.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,14.0,4.0
299998,2010-01-02,WN,2796,LAS,PDX,13:35:00,14:22:00,47.0,20.0,14:42:00,16:44:00,2.0,16:05:00,16:46:00,41.0,0.0,NaN,0.0,150.0,144.0,122.0,762.0,0.0,0.0,4.0,0.0,37.0,NaN,0.0,20.0,2.0
299999,2010-01-02,WN,5836,LAS,PDX,08:10:00,08:10:00,0.0,9.0,08:19:00,10:24:00,4.0,10:35:00,10:28:00,-7.0,0.0,NaN,0.0,145.0,138.0,125.0,762.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,9.0,4.0


In [27]:
# Check column type
# flight_data.info()
checkMissing(flight_data)

           column_name  percentage
0    CANCELLATION_CODE  100.000000
1          Unnamed: 27  100.000000
2        CARRIER_DELAY   74.468158
3            ARR_DELAY    0.177852
4  ACTUAL_ELAPSED_TIME    0.172026
5             AIR_TIME    0.172026
6            DEP_DELAY    0.015078


In [13]:
#check_depart = flight_data[flight_data['DEP_DELAY_c'] != flight_data['DEP_DELAY']]  
#check_arrival = flight_data[flight_data['ARR_DELAY_c'] != flight_data['ARR_DELAY']] 

#print("The duration of delayed which is not tally:")
#print("For time taken of departure delay")
#print(check_depart.shape)
#print()
#print("For time taken of arrival delay")
#print(check_arrival.shape)

In [28]:
checkMissing(flight_data)

           column_name  percentage
0    CANCELLATION_CODE  100.000000
1          Unnamed: 27  100.000000
2        CARRIER_DELAY   74.468158
3            ARR_DELAY    0.177852
4  ACTUAL_ELAPSED_TIME    0.172026
5             AIR_TIME    0.172026
6            DEP_DELAY    0.015078


In [24]:
#check_arrival.loc[:,['ARR_TIME','CRS_ARR_TIME','ARR_DELAY_c','ARR_DELAY']]

flight_data[[  'CANCELLED',
                  'OP_CARRIER_FL_NUM',
                  'CRS_DEP_TIME',
                  'DEP_TIME',
                  'CRS_ARR_TIME',
                  'ARR_TIME',
                  'WHEELS_ON',
                  'WHEELS_OFF']]

,CANCELLED,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,CRS_ARR_TIME,ARR_TIME,WHEELS_ON,WHEELS_OFF
0,0.0,2377,11:35:00,11:44:00,13:00:00,13:02:00,12:53:00,11:56:00
1,0.0,2378,22:25:00,22:20:00,23:25:00,23:15:00,23:11:00,22:34:00
3,0.0,2502,16:55:00,18:05:00,19:00:00,20:05:00,19:41:00,18:11:00
4,0.0,2502,13:20:00,14:40:00,16:25:00,17:35:00,17:30:00,14:49:00
5,0.0,2503,19:25:00,19:09:00,20:30:00,20:53:00,20:00:00,19:17:00
...,...,...,...,...,...,...,...,...
299995,0.0,3674,19:25:00,20:36:00,20:25:00,21:24:00,21:16:00,20:44:00
299996,0.0,5752,09:50:00,10:05:00,10:45:00,10:57:00,10:53:00,10:21:00
299997,0.0,580,19:40:00,20:20:00,22:05:00,22:35:00,22:32:00,20:31:00
299998,0.0,2796,13:35:00,14:22:00,16:05:00,16:46:00,16:44:00,14:42:00


In [16]:
# Drop the unused columns 

flight_data.drop(['Unnamed: 27', 
                  'CANCELLATION_CODE',
                  'CANCELLED',
                  'OP_CARRIER_FL_NUM',
                  'CRS_DEP_TIME',
                  'DEP_TIME',
                  'CRS_ARR_TIME',
                  'ARR_TIME',
                  'WHEELS_ON',
                  'WHEELS_OFF'
                 ],
                 axis = 1, inplace = True)

Imputation with cold-deck method may introduce noise to the data, so to avoid that to interfere the overall performance, Since we will just drop those missing rows as the percentage of missing values is just 1%.

In [17]:
#Check number of missing values
flight_data.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                     0
DEST                       0
DEP_DELAY               4735
TAXI_OUT                   0
TAXI_IN                    0
ARR_DELAY              17560
DIVERTED                   0
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    14962
AIR_TIME               14962
DISTANCE                   0
CARRIER_DELAY              0
WEATHER_DELAY              0
NAS_DELAY                  0
SECURITY_DELAY             0
LATE_AIRCRAFT_DELAY        0
WHEELS_OFF_elapse          0
WHEELS_ON_elapse           0
dtype: int64

In [18]:
flight_data = flight_data.dropna()

In [19]:
#Check number of missing values
flight_data.isna().sum()

FL_DATE                0
OP_CARRIER             0
ORIGIN                 0
DEST                   0
DEP_DELAY              0
TAXI_OUT               0
TAXI_IN                0
ARR_DELAY              0
DIVERTED               0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
CARRIER_DELAY          0
WEATHER_DELAY          0
NAS_DELAY              0
SECURITY_DELAY         0
LATE_AIRCRAFT_DELAY    0
WHEELS_OFF_elapse      0
WHEELS_ON_elapse       0
dtype: int64

#### Rename of OP_CARRIER

`OP_CARRIER` refers to the airline identifier recorded with its abbreviation form. To make the content more intuitive, the column of abbreviation names are transformed with airlines extended names. The main reference source for the extended name is obtained from <a href="https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States">Wikipedia</a>


In [30]:
# Check the instances for OP_CARRIER
flight_data.OP_CARRIER.unique()

array(['AA', 'EV', 'F9', 'FL', 'HA', 'MQ', 'OO', 'VX', 'WN', 'UA', 'US',
       'AS', 'B6', 'DL', 'NK', 'YV', '9E', 'G4', 'OH', 'YX', 'XE', 'CO',
       'NW'], dtype=object)

In [21]:
flight_data['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}, inplace=True)

In [22]:
flight_data.OP_CARRIER.nunique()

18

In [23]:
flight_data.OP_CARRIER.value_counts()

Southwest Airlines    1326376
Delta Airlines         938464
American Airlines      892021
SkyWest Airlines       758717
United Airlines        609226
Republic Airways       303927
JetBlue Airways        293075
Envoy Air              283788
PSA Airlines           264929
Alaska Airlines        240352
Endeavor Air           231211
Mesa Airline           208382
ExpressJet             196072
Spirit Airlines        171359
Frontier Airlines      116058
Allegiant Air           94982
Hawaiian Airlines       83161
Virgin America          17012
Name: OP_CARRIER, dtype: int64

#### Data reduction: Focus on Busy Airports
Even though a lot of pre-processing and cleaning has been done on a separate notebook, this one will be only aiming to obtain a final dataframe for the modeling which will have the top 20 cities only. The reason to do it this way is because the DEST feature is a categorical column of 358 values. The hot encoding will then be done for this 20 instead of the 358 total destinations.  

In [24]:
flight_data.DEST.value_counts().iloc[:20]

ATL    384813
ORD    322119
DFW    271096
DEN    232583
CLT    225450
LAX    217452
SFO    171824
PHX    171433
IAH    170255
LGA    162095
LAS    158721
MSP    156532
DTW    154229
BOS    142239
SEA    137829
EWR    137238
MCO    135272
DCA    127517
JFK    122315
PHL    112410
Name: DEST, dtype: int64